In [1]:
%load_ext autoreload
import f3dasm
from f3dasm_simulate import openFoam
from PyFoam import FoamInformation
from pathlib import Path

2023-06-25 16:32:08,305 - Imported f3dasm (version: 1.1.0)
2023-06-25 16:32:08.744962: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-25 16:32:08.746203: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-25 16:32:08.771413: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-25 16:32:08.772509: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-25 16:32:09.184829: W tensorflow/compiler/tf2t

In [2]:
# Be sure that openFOam is in the path, for instance by running
# 'source /opt/openfoam10/etc/bashrc' before starting the notebook

# The following statement should return openFoam version
print(f"openFoam version: {FoamInformation.foamVersion()}")

openFoam version: 10


In [19]:
%autoreload 2
mesh_size = f3dasm.DiscreteParameter(
    name="mesh_factor", lower_bound=1, upper_bound=10
)
input_velocity = f3dasm.ContinuousParameter(
    name="input_velocity", lower_bound=0.1, upper_bound=10.0
)
output_pressure = f3dasm.ContinuousParameter(
    name="output_pressure", lower_bound=0.0, upper_bound=1.0
)

avg_error = f3dasm.ContinuousParameter(name="avg_error")

design = f3dasm.DesignSpace(
    input_space=[mesh_size, input_velocity, output_pressure],
    output_space=[avg_error],
)

# Create the sampler object
sampler = f3dasm.sampling.RandomUniform(design=design, seed=42)

data: f3dasm.ExperimentData = sampler.get_samples(numsamples=5)

# f3dasm.experiment.quickstart()

job_queue = f3dasm.experiment.JobQueue.from_experimentdata(
    filename="my_jobs", experimentdata=data
)

job_queue.write_new_jobfile()#overwrite=True)
data.store('data')

In [20]:
%autoreload 2

def construct_parameters(names:list, args: dict):
    parameters = {}
    for name in names:
        parameters[name]=dict(value=args[name], description=name)

    return parameters

while True:
    try:
        job_id = job_queue.get()
       
    except f3dasm.experiment.NoOpenJobsError:
        break

    # data = f3dasm.design.("data")
 
    args = data.get_inputdata_by_index(job_id)

    def avg_error(self):
        error_field = self.solution_dir.getDictionaryContents(
            directory="0", name="error"
        )["internalField"].val

        self.results = dict(avg_error=sum(error_field) / len(error_field))
  

    # Create simulation info
    material=openFoam.material.Material()
    geometry=openFoam.geometry.Geometry()
    boundary=openFoam.boundary.Boundary(parameters=construct_parameters(names=["input_velocity", "output_pressure"], args=args))
    mesh=openFoam.mesh.Mesh(parameters=construct_parameters(names=["mesh_factor"], args=args))
    solver=openFoam.solver.Solver()
    simulation_info = openFoam.simulator_info.SimulationInfo(material=material, geometry=geometry, boundary=boundary, mesh=mesh, solver=solver,
                                                             case_source_path=Path("cylinder.orig"),
        job_id=job_id)

    simulator_info = openFoam.simulator_info.SimulatorInfo(fork="",
                                                           version="",
                                                           build_date="",
                                                           preprocessors=[dict(preprocessor="prepareCase", options=["--no-mesh-create"]),
                                                                          dict(preprocessor="blockMesh", options=[]),],
                                                            solvers=[dict(solver="auto",
                                                                          options=["-withFunctionObjects", "-writePhi", "-writep"])],
                                                            postprocessors=[dict(postprocessor="postProcess", options=["-func", "streamFunction"], post_func=avg_error)])

    simulator = openFoam.openfoam_simulator.openFoamSimulator(simulation_info=simulation_info, simulator_info=simulator_info)

    simulator.pre_process(overwrite=True)

    simulator.execute()

    simulator.post_process()

    data.set_outputdata_by_index(job_id, simulator.results["avg_error"])

    data.store("data")

    job_queue.mark_finished(job_id)


data.data

2023-06-25 17:13:42,803 - Starting job cylinder_0 at /home/guillaume/Documents/code/f3dasm_simulate/tutorials/openFoam/cylinder/jobs/cylinder_0
 PyFoam WARNING on line 698 of file /home/guillaume/miniconda3/envs/f3dasm_env/lib/python3.10/site-packages/PyFoam/RunDictionary/SolutionDirectory.py : The first timestep in /home/guillaume/Documents/code/f3dasm_simulate/tutorials/openFoam/cylinder/jobs/cylinder_0  is  None not a number. Doing nothing 
2023-06-25 17:13:42,928 - Starting: blockMesh -case /home/guillaume/Documents/code/f3dasm_simulate/tutorials/openFoam/cylinder/jobs/cylinder_0 in /home/guillaume/Documents/code/f3dasm_simulate/tutorials/openFoam/cylinder
2023-06-25 17:13:42,942 - Started with PID 170075
2023-06-25 17:13:43,551 - Finished
2023-06-25 17:13:43,566 - Automatic mode will use potentialFoam solver.
2023-06-25 17:13:43,567 - Starting: potentialFoam -case /home/guillaume/Documents/code/f3dasm_simulate/tutorials/openFoam/cylinder/jobs/cylinder_0 -withFunctionObjects -write

input                                   output
  mesh_factor input_velocity output_pressure avg_error
0           6       3.807947        0.950714  0.053566
1           5       7.346740        0.598658  0.053693
2           2       1.644585        0.155995  0.054690
3           8       0.675028        0.866176  0.053637
4           6       6.051039        0.708073  0.053566